In [1]:

import os

In [2]:
%pwd

'/Volumes/RRR/Macbook-workdrive/DataScienceEnd2EndProjects/Chicken-Disease-Classification/research'

In [3]:
os.chdir("../")


In [4]:
%pwd

'/Volumes/RRR/Macbook-workdrive/DataScienceEnd2EndProjects/Chicken-Disease-Classification'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [8]:
import os
import urllib.request as request
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    def download_data(self):
        if not os.path.exists(self.config.local_data_file):
            filename, header = request.urlretrieve(
                self.config.source_URL,
                self.config.local_data_file
            )
            logger.info(f"Downloaded {get_size(Path(self.config.local_data_file))} from {self.config.source_URL} to {self.config.local_data_file}")
        else:
            logger.info(f"File {self.config.local_data_file} already exists")
            logger.info(f"File size: {get_size(Path(self.config.local_data_file))}")
    
    def unzip_data(self):
        if not os.path.exists(self.config.unzip_dir):
            os.makedirs(self.config.unzip_dir)
            logger.info(f"Created directory {self.config.unzip_dir}")
        else:
            logger.info(f"Directory {self.config.unzip_dir} already exists")
        
        logger.info(f"Unzipping {self.config.local_data_file} to {self.config.unzip_dir}")
        import zipfile
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(self.config.unzip_dir)
        logger.info(f"Unzipped {self.config.local_data_file} to {self.config.unzip_dir}")
                    

In [10]:
try:
    config=ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion=DataIngestion(data_ingestion_config)
    data_ingestion.download_data()
    data_ingestion.unzip_data()
    logger.info("Data Ingestion Completed")
except Exception as e:
    logger.error(f"Error in Data Ingestion: {e}")
    raise e

[[2024-05-14 20:30:36,116]: INFO: common: yaml file: config/config.yaml loaded successfully]
[[2024-05-14 20:30:36,118]: INFO: common: yaml file: params.yaml loaded successfully]
[[2024-05-14 20:30:36,120]: INFO: common: created directory at: artifacts]
[[2024-05-14 20:30:36,125]: INFO: common: created directory at: artifacts/data_ingestion]
[[2024-05-14 20:30:36,151]: INFO: 1368670523: File artifacts/data_ingestion/data.zip already exists]
[[2024-05-14 20:30:36,165]: INFO: 1368670523: File size: ~ 11345 KB]
[[2024-05-14 20:30:36,179]: INFO: 1368670523: Directory artifacts/data_ingestion already exists]
[[2024-05-14 20:30:36,184]: INFO: 1368670523: Unzipping artifacts/data_ingestion/data.zip to artifacts/data_ingestion]
[[2024-05-14 20:30:36,622]: INFO: 1368670523: Unzipped artifacts/data_ingestion/data.zip to artifacts/data_ingestion]
[[2024-05-14 20:30:36,623]: INFO: 1641388812: Data Ingestion Completed]


'/Volumes/RRR/Macbook-workdrive/DataScienceEnd2EndProjects/Chicken-Disease-Classification'